In [1]:
from tensorflow.python.summary.summary_iterator import summary_iterator
import glob
import numpy as np

'''
this file load the event file and analysis the result for the total 16 experiments.
'''

'\nthis file load the event file and analysis the result for the total 16 experiments.\n'

In [2]:
flag_involve_fake_data_rlt = True

In [3]:
'''
get event list in the checkpoint/poseaug folder
'''
# get the list of all 16 event file
home_dir = './poseaug'
path_lst_to_events_file = glob.glob(home_dir+'/*/*/*/*tfevent*')


In [4]:
print(len(path_lst_to_events_file))

16


In [5]:
# total 16 experiment

dataset_namelst = ['detectron_ft_h36m', 'cpn_ft_h36m_dbb', 'hr', 'gt']
model_namelst = ['mlp', 'gcn', 'stgcn', 'videopose']

# dataset_namelst = ['hr', 'gt']
# model_namelst = ['stgcn', 'videopose']

In [6]:
def get_eventpath_byDM(total_list, d, m):
    '''
    d: dataset name
    m: model name
    '''
    out = []
#     assert len(total_list)==16, 'supposed 16 in total'
    for item in total_list:
        if '/'+d+'/' in item:
            if '/'+m+'/' in item:
                out.append(item)
    assert len(out) == 1, 'supposed only 1 here'
    return out[0]

In [7]:
tag_dict = {
    's911_p1_fake': 'posenet_H36M_test/p1score_fake',
    's911_p1': 'posenet_H36M_test/p1score_real',
    'mpi_p1_fake': 'posenet_mpi3d_loader_flip/p1score_fake',
    'mpi_p1': 'posenet_mpi3d_loader_flip/p1score_real',
}

# tag_keylst = ['mpi_p1', 's911_p1']
tag_keylst = ['s911_p1','mpi_p1_fake', 'mpi_p1']

In [8]:
'''
extract the value from tag in event file
'''
            
def get_log(event_path, tag):
    try:
        a = {}
        a[tag] = []
        a['step']=[]
        for e in summary_iterator(event_path):
            for v in e.summary.value:
                if v.tag == tag:
                    a['step'].append(e.step)
                    a[tag].append(v.simple_value)
                
        return a
    
    # Dirty catch of DataLossError
    except:
        print('Event file possibly corrupt: {}'.format(event_path))
        return None
            

In [9]:
# organise into a dict

rlt_summary = {}

# init
for dataset_name in dataset_namelst:
    print('now work on: ', dataset_name)
    if not dataset_name in rlt_summary:
        rlt_summary[dataset_name] = {}
    for model_name in model_namelst:
        print('now work on: ', dataset_name, model_name)
        if not model_name in rlt_summary[dataset_name]:
            rlt_summary[dataset_name][model_name] = {}
        # extract file list from path
        rlt_summary[dataset_name][model_name]['event_path']=get_eventpath_byDM(path_lst_to_events_file
                                                                             , dataset_name, model_name)
        
        for tag_key in tag_keylst:
            # get data from tag_key
            tmp_tag = tag_dict[tag_key]
            tmp_log = get_log(rlt_summary[dataset_name][model_name]['event_path'], tmp_tag)
            rlt_summary[dataset_name][model_name][tag_key] = tmp_log[tmp_tag]
            if 'step' in rlt_summary[dataset_name][model_name]:
                assert rlt_summary[dataset_name][model_name]['step'] == tmp_log['step']
            else:
                rlt_summary[dataset_name][model_name]['step'] = tmp_log['step']


now work on:  detectron_ft_h36m
now work on:  detectron_ft_h36m mlp
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
now work on:  detectron_ft_h36m gcn
now work on:  detectron_ft_h36m stgcn
now work on:  detectron_ft_h36m videopose
now work on:  cpn_ft_h36m_dbb
now work on:  cpn_ft_h36m_dbb mlp
now work on:  cpn_ft_h36m_dbb gcn
now work on:  cpn_ft_h36m_dbb stgcn
now work on:  cpn_ft_h36m_dbb videopose
now work on:  hr
now work on:  hr mlp
now work on:  hr gcn
now work on:  hr stgcn
now work on:  hr videopose
now work on:  gt
now work on:  gt mlp
now work on:  gt gcn
now work on:  gt stgcn
now work on:  gt videopose


In [10]:
def get_min(score_lst):
    out={}
    idx = np.argmin(score_lst)
    score = score_lst[idx]
    return {'idx': idx, 'score': round(score,1), 'base':round(score_lst[0], 1), 'improve':-round(score - score_lst[0], 1)}

In [11]:
'''check the p1 result with/without poseaug'''

'check the p1 result with/without poseaug'

In [12]:
# here take every result and get the best one, save in the dict.
for dataset_name in dataset_namelst:
    print('now work on: ', dataset_name)
    for model_name in model_namelst:
#         print('now work on: ', dataset_name, model_name)
        rlt_summary[dataset_name][model_name]['best_s911_p1'] = get_min(
            rlt_summary[dataset_name][model_name]['s911_p1'])
        rlt_summary[dataset_name][model_name]['best_mpi_p1'] = get_min(
            rlt_summary[dataset_name][model_name]['mpi_p1'])
        rlt_summary[dataset_name][model_name]['best_mpi_p1_fake'] = get_min(
            rlt_summary[dataset_name][model_name]['mpi_p1_fake'])

now work on:  detectron_ft_h36m
now work on:  cpn_ft_h36m_dbb
now work on:  hr
now work on:  gt


In [13]:
'''
now start use panda to view the result
'''

'\nnow start use panda to view the result\n'

In [14]:
# organise into a dict

view_dict = {
    'best_s911_p1':{},
    'best_mpi_p1':{},
    'best_mpi_p1_fake':{},
}

for view_opinion in view_dict:
    # init
    base_summary = {}
    aug_summary = {}
    for dataset_name in dataset_namelst:
        print('now work on: ', dataset_name)
        if not dataset_name in aug_summary:
            base_summary[dataset_name] = {}
            aug_summary[dataset_name] = {}
        for model_name in model_namelst:
#             print('now work on: ', dataset_name, model_name)
            base_summary[dataset_name][model_name] = rlt_summary[dataset_name][model_name][view_opinion]['base']
            aug_summary[dataset_name][model_name] = rlt_summary[dataset_name][model_name][view_opinion]['score']

    view_dict[view_opinion] = {
        'baseline': base_summary,
        'poseaug': aug_summary,
    }


now work on:  detectron_ft_h36m
now work on:  cpn_ft_h36m_dbb
now work on:  hr
now work on:  gt
now work on:  detectron_ft_h36m
now work on:  cpn_ft_h36m_dbb
now work on:  hr
now work on:  gt
now work on:  detectron_ft_h36m
now work on:  cpn_ft_h36m_dbb
now work on:  hr
now work on:  gt


In [15]:
if flag_involve_fake_data_rlt:
    for dataset_name in dataset_namelst:
        for model_name in model_namelst:
            view_dict['best_mpi_p1']['poseaug'][dataset_name][model_name] = \
            min(view_dict['best_mpi_p1_fake']['poseaug'][dataset_name][model_name],\
                view_dict['best_mpi_p1']['poseaug'][dataset_name][model_name])

In [16]:
import pandas as pd
from IPython.display import display

for view_opinion in ['best_s911_p1', 'best_mpi_p1']:
    print('==========================================')
    print('view_opinion: ', view_opinion)
    
    for view_key in ['baseline', 'poseaug']:
        print('------------------------------------------')
        print('view_key: ', view_key)
        display(pd.DataFrame.from_dict(view_dict[view_opinion][view_key]))
#         print('')


view_opinion:  best_s911_p1
------------------------------------------
view_key:  baseline


,detectron_ft_h36m,cpn_ft_h36m_dbb,hr,gt
mlp,60.3,55.1,51.5,41.4
gcn,67.3,61.8,56.5,44.6
stgcn,62.5,56.4,53.7,41.9
videopose,60.9,55.9,52.2,41.5


------------------------------------------
view_key:  poseaug


,detectron_ft_h36m,cpn_ft_h36m_dbb,hr,gt
mlp,59.1,53.5,51.3,38.4
gcn,65.5,60.5,55.4,41.0
stgcn,60.8,54.7,51.5,36.4
videopose,58.5,53.4,50.5,38.6


view_opinion:  best_mpi_p1
------------------------------------------
view_key:  baseline


,detectron_ft_h36m,cpn_ft_h36m_dbb,hr,gt
mlp,92.3,89.2,85.7,83.1
gcn,108.7,100.0,96.1,97.7
stgcn,95.9,92.4,84.5,87.6
videopose,93.4,91.1,85.9,85.4


------------------------------------------
view_key:  poseaug


,detectron_ft_h36m,cpn_ft_h36m_dbb,hr,gt
mlp,81.4,80.0,74.8,73.8
gcn,91.7,90.1,77.9,80.0
stgcn,74.7,75.2,77.9,78.0
videopose,79.9,77.2,75.1,72.4
